<a href="https://colab.research.google.com/github/nakanoelio/i2a2-challenge-petr4-trad-sys/blob/main/I2A2_PETR4_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install --upgrade mplfinance

#Instalação da Biblioteca TA-lib
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

     |████████████████████████████████| 6.3MB 4.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=ffc0ec952f3ef1e7747c693e09774c8ce5f2d3732a36265c70048bb58b581f73
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 71kB 3.0MB/s 
Selecting previously unselected package libta-lib0.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing trig

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sn
import matplotlib.pyplot as plt

import yfinance as yf

import talib as ta

from sklearn import metrics
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.linear_model import LinearRegression

In [ ]:
stock_ticker = 'PETR4.SA'
start_date = '2015-11-30'
end_date = '2021-5-27'

yf_petr4 = yf.Ticker(stock_ticker)
df_petr4 = yf_petr4.history(start=start_date, end=end_date)

stock_ticker = '^BVSP'
yf_ibov = yf.Ticker(stock_ticker)
df_ibov = yf_ibov.history(start=start_date, end=end_date)

In [ ]:
df_ibov

In [ ]:
df_petr4

In [ ]:
df_petr4_beta = pd.concat([df_ibov["Close"].pct_change(),df_petr4["Close"].pct_change()],axis=1,ignore_index=True)

df_petr4_beta.columns=["Close_IBOV","Close_PETR4"]

df_petr4_beta["Beta"] = df_petr4_beta["Close_PETR4"].rolling(30).cov(df_petr4_beta["Close_IBOV"].rolling(30))/df_petr4_beta["Close_IBOV"].rolling(30).var()

df_petr4_beta["Beta_expected_PETR4"] = df_petr4_beta["Close_IBOV"]*df_petr4_beta["Beta"]

df_petr4_beta["PETR4_Excess_Variat"] = (df_petr4_beta["Close_PETR4"] - df_petr4_beta["Beta_expected_PETR4"])/df_petr4_beta["Beta_expected_PETR4"]

var_tolerance = df_petr4_beta["Close_IBOV"].var()/df_petr4_beta["Close_IBOV"].mean()

df_petr4_beta.fillna(0,inplace=True)
df_petr4_beta.loc[df_petr4_beta["PETR4_Excess_Variat"]  > var_tolerance, "PETR4_Excess_Variat_Disc"] = 3
df_petr4_beta.loc[df_petr4_beta["PETR4_Excess_Variat"] <= var_tolerance , "PETR4_Excess_Variat_Disc"] = 2
df_petr4_beta.loc[df_petr4_beta["PETR4_Excess_Variat"] >= -var_tolerance , "PETR4_Excess_Variat_Disc"] = 2
df_petr4_beta.loc[df_petr4_beta["PETR4_Excess_Variat"]  < -var_tolerance, "PETR4_Excess_Variat_Disc"] = 1

In [ ]:
df_petr4_beta

,Close_IBOV,Close_PETR4,Beta,Beta_expected_PETR4,PETR4_Excess_Variat,PETR4_Excess_Variat_Disc
Date,,,,,,
2015-11-30,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
2015-12-01,-0.001618,-0.023468,0.000000,0.000000,0.000000,2.0
2015-12-02,-0.002930,0.004005,0.000000,0.000000,0.000000,2.0
2015-12-03,0.032907,0.061170,0.000000,0.000000,0.000000,2.0
2015-12-04,-0.022245,-0.057644,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...,...
2021-05-20,0.000530,-0.008413,1.414554,0.000750,-12.221082,1.0
2021-05-21,-0.000888,0.000771,1.455016,-0.001293,-1.596748,1.0
2021-05-24,0.011746,0.016956,1.453604,0.017074,-0.006958,2.0
